## Download Libraries

In [17]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
!ollama pull nomic-embed-text

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                    

## Libraries

In [19]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
llama3:latest          	a6990ed6be41	4.7 GB	16 minutes ago        	
mistral:latest         	61e88e884507	4.1 GB	3 days ago            	
nomic-embed-text:latest	0a109f422b47	274 MB	Less than a second ago	


In [20]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

## Load PDF

In [21]:
local_path = "monopoly.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

## Vector Embeddings

In [23]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [24]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 4/4 [00:09<00:00,  2.35s/it]


## Retrieval

In [25]:
# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model)

In [26]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [27]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [28]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [29]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


'Based on the provided context, it appears that the primary objective of this game (likely Monopoly) is to become wealthy by buying, selling, and trading properties, as well as collecting rent from other players who land on them. The game seems to involve strategic decision-making regarding property acquisition, management, and improvement (through building houses and hotels), as well as negotiations with other players to buy, sell, or trade properties. Ultimately, the goal is likely to bankrupt all other players and emerge as the wealthiest player remaining in the game.'

In [30]:
chain.invoke("file ini membahas mengenai apa?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]


'Based on the given context, it appears that the document (PDF) is about the rules of the classic board game "Monopoly". The text describes various aspects of the game, including building houses and hotels, selling properties, mortgaging properties, and bankruptcy.'

In [31]:
# vector_db.delete_collection()